# Tarea M10 MPAD de González_Ramón

# Problema machine learning: Análisis jugadores similares mediante Clustering
**K-means para índices generales, seguido de KNN como técnica complementaria dentro del Clúster más relevante**


### Template para Desarrollo de Modelos siguiendo CRISP-DM

In [2]:
### Configuración inicial, importación de bibliotecas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline


# Configuraciones
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# 1. Comprensión del Negocio


### <u>Objetivo del proyecto</u>

El objetivo de este proyecto es desarrollar un sistema de análisis de jugadores similares para comparar con los de la actual plantilla del Atlético de Madrid, y que permita identificar perfiles similares por una posible salida al mercado para sustituir o añadir a la plantilla. Esta herramienta será útil para la planificación de plantilla, detección de talentos y, como hemos dicho, posibles reemplazos en caso de lesiones o transferencias.

### <u>Contexto deportivo</u>

El fútbol moderno está cada vez más dominado por el análisis de datos. Esta es una fórmula en la que los equipos se pueden apoyar intentando buscar ventajas competitivas a través del uso inteligente de información estadística para la toma de decisiones. El Atlético de Madrid, como club de élite, necesita herramientas avanzadas para gestionar su plantilla y planificar el mercado de fichajes, que complementen otras vías para el conocimiento del posible jugador atlético.

### <u>Descripción del problema</u>

Se requiere un sistema que, mediante técnicas de machine learning, pueda:
- Agrupar jugadores según perfiles estadísticos similares (clustering);
- Dentro de los grupos formados, encontrar jugadores específicamente similares a los del Atlético de Madrid (KNN);
- Considerar diferentes métricas según la posición del jugador (porteros, defensas, centrocampistas, delanteros);
- Integrar datos de rendimiento deportivo con información de mercado, o métricas avanzadas (valor, contrato, etc.).

### <u>Criterios de éxito</u>

- Crear grupos de jugadores con sentido deportivo (clusters interpretables);
- Identificar jugadores similares a los actuales del Atlético de Madrid;
- Integrar correctamente métricas específicas por posición;
- Proporcionar recomendaciones útiles para departamento deportivo.

### <u>Recursos disponibles</u>

- Datos estadísticos de las 5 grandes ligas europeas (temporada 24/25);
- Datos específicos para porteros de esas mismas ligas;
- Información de mercado de los jugadores (valor, contrato, etc.);
- Conocimiento previo sobre las necesidades del Atlético de Madrid;
- Archivos master para mapear posibles id, fotos, o cualquier recurso necesario.

# 2. Comprensión de los Datos

## 2.1 Carga de Datos
'''
TODO: 
- Cargar datasets
- Describir fuentes de datos
- Analizar estructura inicial
'''

In [ ]:
def cargar_datos():
    """
    Carga los datasets necesarios para el análisis
    """
    # Cargar datos generales de jugadores de campo
    df_general = pd.read_csv('stats_big5_24_25.csv')
    
    # Cargar datos específicos de porteros
    df_porteros = pd.read_csv('stats_big5_gk_24_25.csv')
    
    # Cargar datos de mercado para análisis KNN
    df_mercado = pd.read_csv('knn_players.csv')
    
    # Cargar datos de jugadores del Atlético de Madrid
    df_atletico = pd.read_csv('data/master/jugadores_master.csv')
    
    return df_general, df_porteros, df_mercado, df_atletico

# Cargar datos
try:
    df_general, df_porteros, df_mercado, df_atletico = cargar_datos()
    print("Datos cargados correctamente")
except Exception as e:
    print(f"Error al cargar los datos: {e}")
    # Crear datos de ejemplo para desarrollo
    print("Creando datos de ejemplo para desarrollo")
    
    # Ejemplo de datos para desarrollo
    # Creamos datasets con las columnas que hemos visto en los ejemplos compartidos
    
    # Datos generales de ejemplo
    df_general = pd.DataFrame()
    
    # Datos de porteros de ejemplo
    df_porteros = pd.DataFrame()
    
    # Datos de mercado de ejemplo
    df_mercado = pd.DataFrame()
    
    # Datos del Atlético de Madrid
    df_atletico = pd.DataFrame()


## 2.2 Análisis Exploratorio


In [ ]:
def analisis_exploratorio(df):
    """
    Realizar análisis exploratorio inicial
    """
    # Información básica
    print("Información del Dataset:")
    print(df.info())
    
    # Estadísticas descriptivas
    print("\nEstadísticas Descriptivas:")
    print(df.describe())
    
    # Valores faltantes
    print("\nValores Faltantes:")
    print(df.isnull().sum())
    
    return None

In [ ]:
def analisis_exploratorio(df, title="Dataset"):
    """
    Realizar análisis exploratorio inicial
    """
    print(f"\n{'-'*20} Análisis de {title} {'-'*20}")
    
    # Información básica
    print(f"\nDimensiones del Dataset: {df.shape}")
    
    # Primeras filas
    print("\nPrimeras filas:")
    display(df.head(3))
    
    # Tipos de datos
    print("\nTipos de datos:")
    print(df.dtypes.value_counts())
    
    # Valores faltantes
    missing_values = df.isnull().sum()
    print("\nColumnas con valores faltantes:")
    print(missing_values[missing_values > 0])
    
    # Estadísticas descriptivas
    print("\nEstadísticas descriptivas de columnas numéricas:")
    display(df.describe().T)
    
    return None

# Análisis de los datasets
if not df_general.empty:
    analisis_exploratorio(df_general, "Jugadores de Campo")
if not df_porteros.empty:
    analisis_exploratorio(df_porteros, "Porteros")
if not df_mercado.empty:
    analisis_exploratorio(df_mercado, "Datos de Mercado")
if not df_atletico.empty:
    analisis_exploratorio(df_atletico, "Jugadores del Atlético de Madrid")

## 2.3 Visualizaciones Iniciales

In [ ]:
def visualizaciones_iniciales(df):
    """
    Crear visualizaciones exploratorias
    """
    # Implementar visualizaciones relevantes
    pass

In [ ]:
def visualizaciones_iniciales(df, title="Dataset"):
    """
    Crear visualizaciones exploratorias
    """
    plt.figure(figsize=(12, 10))
    plt.suptitle(f'Visualizaciones exploratorias - {title}', fontsize=16)
    
    # 1. Distribución de posiciones
    if 'Posicion' in df.columns:
        plt.subplot(2, 2, 1)
        posiciones = df['Posicion'].value_counts()
        sns.barplot(x=posiciones.index, y=posiciones.values)
        plt.title('Distribución de Posiciones')
        plt.ylabel('Cantidad de Jugadores')
        plt.xticks(rotation=45)
    
    # 2. Distribución de equipos
    if 'Equipo' in df.columns:
        plt.subplot(2, 2, 2)
        equipos = df['Equipo'].value_counts().head(10)  # Top 10 equipos
        sns.barplot(x=equipos.values, y=equipos.index)
        plt.title('Top 10 Equipos por Cantidad de Jugadores')
        plt.xlabel('Cantidad de Jugadores')
    
    # 3. Distribución de minutos jugados
    if 'Minutos' in df.columns:
        plt.subplot(2, 2, 3)
        sns.histplot(df['Minutos'], bins=20, kde=True)
        plt.title('Distribución de Minutos Jugados')
        plt.xlabel('Minutos')
    
    # 4. Correlación entre goles y asistencias (para jugadores de campo)
    if 'Goles' in df.columns and 'Asistencias' in df.columns:
        plt.subplot(2, 2, 4)
        sns.scatterplot(x='Goles', y='Asistencias', hue='Posicion', data=df)
        plt.title('Goles vs Asistencias por Posición')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()
    
    # Si hay datos numéricos, mostrar matriz de correlación
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    if len(numeric_cols) > 5:  # Solo si hay suficientes columnas numéricas
        plt.figure(figsize=(14, 12))
        corr_matrix = df[numeric_cols].iloc[:, :10].corr()  # Usando primeras 10 columnas numéricas para mayor claridad
        sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
        plt.title(f'Matriz de Correlación - {title} (Primeras 10 columnas numéricas)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    
    return None

# Visualizaciones de los datasets
if not df_general.empty:
    visualizaciones_iniciales(df_general, "Jugadores de Campo")
if not df_porteros.empty:
    visualizaciones_iniciales(df_porteros, "Porteros")

# 3. Preparación de Datos

## 3.1 Limpieza de Datos

In [ ]:
def limpiar_datos(df):
    """
    Realizar limpieza de datos
    """
    # Implementar limpieza
    pass

In [ ]:
def limpiar_datos(df_general, df_porteros, df_mercado, df_atletico):
    """
    Realizar limpieza de datos y preparación inicial
    """
    # Copias para no modificar los originales
    df_general_clean = df_general.copy() if not df_general.empty else pd.DataFrame()
    df_porteros_clean = df_porteros.copy() if not df_porteros.empty else pd.DataFrame()
    df_mercado_clean = df_mercado.copy() if not df_mercado.empty else pd.DataFrame()
    df_atletico_clean = df_atletico.copy() if not df_atletico.empty else pd.DataFrame()
    
    # Filtrar jugadores con minutos mínimos (por ejemplo, 400 minutos)
    if 'Minutos' in df_general_clean.columns:
        df_general_clean = df_general_clean[df_general_clean['Minutos'] >= 400]
    
    if 'Minutos' in df_porteros_clean.columns:
        df_porteros_clean = df_porteros_clean[df_porteros_clean['Minutos'] >= 400]
    
    # Manejo de valores faltantes
    # Para datos numéricos: reemplazar con mediana por posición
    if not df_general_clean.empty:
        numeric_cols = df_general_clean.select_dtypes(include=['float64', 'int64']).columns
        df_general_clean[numeric_cols] = df_general_clean.groupby('Posicion')[numeric_cols].transform(
            lambda x: x.fillna(x.median())
        )
    
    if not df_porteros_clean.empty:
        numeric_cols = df_porteros_clean.select_dtypes(include=['float64', 'int64']).columns
        df_porteros_clean[numeric_cols] = df_porteros_clean[numeric_cols].fillna(df_porteros_clean[numeric_cols].median())
    
    # Eliminar duplicados si los hay
    if not df_general_clean.empty:
        df_general_clean = df_general_clean.drop_duplicates(subset=['Jugador', 'Equipo'])
    
    if not df_porteros_clean.empty:
        df_porteros_clean = df_porteros_clean.drop_duplicates(subset=['Jugador', 'Equipo'])
    
    # Limpiar datos de mercado si existen
    if not df_mercado_clean.empty:
        # Convertir valores monetarios a numéricos
        if 'valor_mercado' in df_mercado_clean.columns:
            # Eliminar símbolos y convertir a formato numérico
            df_mercado_clean['valor_mercado'] = df_mercado_clean['valor_mercado'].str.replace('[^\d,.]', '', regex=True)
            df_mercado_clean['valor_mercado'] = pd.to_numeric(df_mercado_clean['valor_mercado'], errors='coerce')
    
    return df_general_clean, df_porteros_clean, df_mercado_clean, df_atletico_clean

# Aplicar limpieza
df_general_clean, df_porteros_clean, df_mercado_clean, df_atletico_clean = limpiar_datos(
    df_general, df_porteros, df_mercado, df_atletico
)

print("Limpieza de datos completada")

## 3.2 Feature Engineering

In [ ]:
def crear_features(df):
    """
    Crear nuevas características
    """
    # Implementar feature engineering
    pass

In [ ]:
def crear_features(df_general, df_porteros):
    """
    Crear nuevas características relevantes para el análisis
    """
    # Hacer copia para no modificar los originales
    df_general_fe = df_general.copy() if not df_general.empty else pd.DataFrame()
    df_porteros_fe = df_porteros.copy() if not df_porteros.empty else pd.DataFrame()
    
    # Características para jugadores de campo
    if not df_general_fe.empty:
        # 1. Efectividad en ataque (goles + asistencias por 90 minutos)
        if all(col in df_general_fe.columns for col in ['Goles', 'Asistencias', 'Minutos']):
            df_general_fe['Efectividad_Ataque'] = (df_general_fe['Goles'] + df_general_fe['Asistencias']) / (df_general_fe['Minutos'] / 90)
        
        # 2. Eficiencia de pases
        if all(col in df_general_fe.columns for col in ['Pases_completados', 'Pases_intentados']):
            df_general_fe['Eficiencia_Pases'] = df_general_fe['Pases_completados'] / df_general_fe['Pases_intentados']
        
        # 3. Contribución defensiva (entradas + intercepciones por 90 minutos)
        if all(col in df_general_fe.columns for col in ['Entradas', 'Intercepciones', 'Minutos']):
            df_general_fe['Contribucion_Defensiva'] = (df_general_fe['Entradas'] + df_general_fe['Intercepciones']) / (df_general_fe['Minutos'] / 90)
        
        # 4. Creación de ocasiones por 90 minutos
        if all(col in df_general_fe.columns for col in ['Pases_clave', 'Minutos']):
            df_general_fe['Creacion_Ocasiones'] = df_general_fe['Pases_clave'] / (df_general_fe['Minutos'] / 90)
            
        # 5. Precisión de disparo
        if all(col in df_general_fe.columns for col in ['Disparos_porteria', 'Disparos']):
            df_general_fe['Precision_Disparo'] = df_general_fe['Disparos_porteria'] / df_general_fe['Disparos']
            df_general_fe['Precision_Disparo'] = df_general_fe['Precision_Disparo'].fillna(0)
        
        # 6. Eficiencia ofensiva (goles / xG)
        if all(col in df_general_fe.columns for col in ['Goles', 'xG']):
            df_general_fe['Eficiencia_Ofensiva'] = df_general_fe['Goles'] / df_general_fe['xG']
            df_general_fe['Eficiencia_Ofensiva'] = df_general_fe['Eficiencia_Ofensiva'].fillna(1)
        
        # 7. Polivalencia posicional (basada en heatmap de posiciones)
        if 'Posicion_princ' in df_general_fe.columns and 'Posicion_2' in df_general_fe.columns:
            df_general_fe['Polivalencia'] = np.where(
                df_general_fe['Posicion_princ'] != df_general_fe['Posicion_2'],
                1, 0
            )
    
    # Características para porteros
    if not df_porteros_fe.empty:
        # 1. Eficiencia de paradas
        if all(col in df_porteros_fe.columns for col in ['Paradas', 'Disparos_recibidos_porteria']):
            df_porteros_fe['Eficiencia_Paradas'] = df_porteros_fe['Paradas'] / df_porteros_fe['Disparos_recibidos_porteria']
            df_porteros_fe['Eficiencia_Paradas'] = df_porteros_fe['Eficiencia_Paradas'].fillna(0)
        
        # 2. Porterías a cero por partido
        if all(col in df_porteros_fe.columns for col in ['Porterías_cero', 'Partidos']):
            df_porteros_fe['Porcentaje_Porterias_Cero'] = df_porteros_fe['Porterías_cero'] / df_porteros_fe['Partidos']
        
        # 3. Eficiencia de pases para porteros
        if all(col in df_porteros_fe.columns for col in ['Pases_completados', 'Pases_intentados']):
            df_porteros_fe['Eficiencia_Pases_Portero'] = df_porteros_fe['Pases_completados'] / df_porteros_fe['Pases_intentados']
        
        # 4. Rendimiento vs esperado (PSxG+/-)
        if 'PSxG+/-' in df_porteros_fe.columns:
            df_porteros_fe['Rendimiento_vs_Esperado'] = df_porteros_fe['PSxG+/-']
        
        # 5. Actividad fuera del área
        if all(col in df_porteros_fe.columns for col in ['Acciones_fuera_area', 'Minutos']):
            df_porteros_fe['Actividad_Fuera_Area'] = df_porteros_fe['Acciones_fuera_area'] / (df_porteros_fe['Minutos'] / 90)
    
    return df_general_fe, df_porteros_fe

# Aplicar feature engineering
df_general_fe, df_porteros_fe = crear_features(df_general_clean, df_porteros_clean)

print("Feature engineering completado")


## 3.3 Preparación para Modelado

In [ ]:
def preparar_datos_modelado(df):
    """
    Preparación final para modelado
    """
    # Implementar preparación final
    pass

In [ ]:
def preparar_datos_modelado(df_general, df_porteros, df_atletico):
    """
    Preparación final para modelado, incluyendo:
    - Selección de características por posición
    - Normalización de datos
    - Creación de datasets específicos para clustering
    """
    # Selección de características relevantes por posición
    if df_general.empty and df_porteros.empty:
        print("No hay datos disponibles para modelado")
        return None, None, None, None
    
    # Crear diccionarios para almacenar datasets por posición
    position_dfs = {}
    scaled_dfs = {}
    atletico_players = {}
    feature_cols = {}
    
    # Variables para debugging
    debugging_info = {}
    
    # Preparar datos para cada posición en jugadores de campo
    if not df_general.empty:
        # Definir características relevantes por posición
        position_features = {
            'Delantero': [
                'Goles', 'Disparos', 'Disparos_porteria', 'xG', 'Pases_area', 'Conducciones_progresivas',
                'Regates_exitosos', 'Precisión_Disparo', 'Eficiencia_Ofensiva', 'Efectividad_Ataque'
            ],
            'Mediocampista': [
                'Asistencias', 'Pases_clave', 'Pases_progresivos', 'Pases_completados', 'Pases_filtrados',
                'Pct_pases', 'Regates_exitosos', 'Entradas', 'Intercepciones', 'Eficiencia_Pases',
                'Contribucion_Defensiva', 'Creacion_Ocasiones'
            ],
            'Defensa': [
                'Entradas', 'Entradas_canadas', 'Intercepciones', 'Despejes', 'Bloqueos',
                'Duelos_aereos_ganados', 'Pct_duelos_aereos', 'Pases_completados',
                'LongPassCmp', 'Contribucion_Defensiva', 'Eficiencia_Pases'
            ]
        }
        
        # Mapeo de posiciones al grupo correspondiente
        position_mapping = {
            'FW': 'Delantero',
            'DF': 'Defensa',
            'MF': 'Mediocampista',
            # Añadir más mapeos según sea necesario
        }
        
        # Si no existe la columna 'Grupo_Posicion', crearla basada en posición principal
        if 'Posicion_princ' in df_general.columns:
            df_general['Grupo_Posicion'] = df_general['Posicion_princ'].apply(
                lambda x: next((v for k, v in position_mapping.items() if k in x), 'Otro')
            )
        elif 'Posicion' in df_general.columns:
            df_general['Grupo_Posicion'] = df_general['Posicion'].apply(
                lambda x: next((v for k, v in position_mapping.items() if k in x), 'Otro')
            )
        
        # Procesar cada grupo de posición
        for position, features in position_features.items():
            # Filtrar jugadores por posición
            pos_df = df_general[df_general['Grupo_Posicion'] == position].copy()
            
            # Si hay datos para esta posición
            if not pos_df.empty:
                # Seleccionar solo las características disponibles en el dataset
                available_features = [f for f in features if f in pos_df.columns]
                
                # Almacenar lista de características utilizadas para referencia
                feature_cols[position] = available_features
                
                # Si no hay suficientes características, usar algunas básicas que deberían estar disponibles
                if len(available_features) < 5:
                    print(f"Advertencia: Pocas características disponibles para {position}, usando características genéricas")
                    available_features = [col for col in ['Minutos', 'Goles', 'Asistencias', 'Pases_completados', 'Entradas'] 
                                          if col in pos_df.columns]
                
                # Debugging
                debugging_info[position] = {
                    'total_players': len(pos_df),
                    'features_used': available_features
                }
                
                # Seleccionar solo las columnas necesarias 
                pos_features_df = pos_df[['Jugador', 'Equipo', 'Nacionalidad'] + available_features].copy()
                
                # Almacenar datasets completos
                position_dfs[position] = pos_features_df
                
                # Identificar jugadores del Atlético de Madrid
                if 'Equipo' in pos_features_df.columns:
                    atletico_players[position] = pos_features_df[pos_features_df['Equipo'] == 'Atletico Madrid'].copy()
                
                # Escalar características
                scaler = StandardScaler()
                features_scaled = scaler.fit_transform(pos_features_df[available_features])
                scaled_df = pd.DataFrame(features_scaled, columns=available_features)
                scaled_df['Jugador'] = pos_features_df['Jugador']
                scaled_df['Equipo'] = pos_features_df['Equipo']
                scaled_df['Nacionalidad'] = pos_features_df['Nacionalidad']
                
                # Almacenar datasets escalados
                scaled_dfs[position] = scaled_df
    
    # Preparar datos para porteros
    if not df_porteros.empty:
        # Definir características relevantes para porteros
        portero_features = [
            'Paradas', 'Pct_paradas', 'Porterías_cero', 'Goles_recibidos_90min',
            'PSxG', 'PSxG+/-', 'Acciones_fuera_area', 'Eficiencia_Paradas',
            'Porcentaje_Porterias_Cero', 'Eficiencia_Pases_Portero', 'Rendimiento_vs_Esperado',
            'Actividad_Fuera_Area'
        ]
        
        # Seleccionar solo las características disponibles
        available_features = [f for f in portero_features if f in df_porteros.columns]
        
        # Almacenar lista de características utilizadas
        feature_cols['Portero'] = available_features
        
        # Si no hay suficientes características, usar algunas básicas
        if len(available_features) < 5:
            print("Advertencia: Pocas características disponibles para Porteros, usando características genéricas")
            available_features = [col for col in ['Minutos', 'Paradas', 'Goles_recibidos', 'Porterías_cero'] 
                                  if col in df_porteros.columns]
        
        # Debugging
        debugging_info['Portero'] = {
            'total_players': len(df_porteros),
            'features_used': available_features
        }
        
        # Seleccionar columnas necesarias
        portero_features_df = df_porteros[['Jugador', 'Equipo', 'Nacionalidad'] + available_features].copy()
        
        # Almacenar dataset completo
        position_dfs['Portero'] = portero_features_df
        
        # Identificar porteros del Atlético de Madrid
        if 'Equipo' in portero_features_df.columns:
            atletico_players['Portero'] = portero_features_df[portero_features_df['Equipo'] == 'Atletico Madrid'].copy()
        
        # Escalar características
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(portero_features_df[available_features])
        scaled_df = pd.DataFrame(features_scaled, columns=available_features)
        scaled_df['Jugador'] = portero_features_df['Jugador']
        scaled_df['Equipo'] = portero_features_df['Equipo']
        scaled_df['Nacionalidad'] = portero_features_df['Nacionalidad']
        
        # Almacenar dataset escalado
        scaled_dfs['Portero'] = scaled_df
    
    # Imprimir información de debugging
    for position, info in debugging_info.items():
        print(f"\nInformación para {position}:")
        print(f"  - Total de jugadores: {info['total_players']}")
        print(f"  - Características utilizadas ({len(info['features_used'])}): {', '.join(info['features_used'])}")
        
        if position in atletico_players and not atletico_players[position].empty:
            print(f"  - Jugadores del Atlético de Madrid: {len(atletico_players[position])}")
        else:
            print(f"  - No se encontraron jugadores del Atlético de Madrid para esta posición")
    
    return position_dfs, scaled_dfs, atletico_players, feature_cols

# Preparar datos para modelado
position_dfs, scaled_dfs, atletico_players, feature_cols = preparar_datos_modelado(
    df_general_fe, df_porteros_fe, df_atletico_clean
)


# 4. Modelado

## 4.1 Primer Modelo (Clustering con K-Means, continuación)

In [ ]:
def entrenar_modelo(X_train, y_train):
    """
    Entrenar primer modelo
    """
    # Implementar entrenamiento
    pass

In [ ]:
def determinar_kmeans_optimo(X, max_clusters=10):
    """
    Determina el número óptimo de clusters usando el método del codo y silhouette score
    """
    if X.shape[0] < max_clusters:
        max_clusters = X.shape[0] - 1
    
    if max_clusters < 2:
        return 2  # Mínimo 2 clusters
    
    inertias = []
    silhouette_scores = []
    
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
        
        # Calcular silhouette score
        if X.shape[0] > k:  # Necesitamos más muestras que clusters
            silhouette_avg = silhouette_score(X, kmeans.labels_)
            silhouette_scores.append(silhouette_avg)
    
    # Visualizar método del codo
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(2, max_clusters + 1), inertias, 'bo-')
    plt.xlabel('Número de clusters')
    plt.ylabel('Inercia')
    plt.title('Método del codo')
    
    plt.subplot(1, 2, 2)
    if silhouette_scores:  # Verificar que tengamos scores
        plt.plot(range(2, max_clusters + 1), silhouette_scores, 'ro-')
        plt.xlabel('Número de clusters')
        plt.ylabel('Silhouette Score')
        plt.title('Silhouette Score por número de clusters')
    
    plt.tight_layout()
    plt.show()
    
    # Determinar el número óptimo
    if silhouette_scores:
        optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2
        print(f"Número óptimo de clusters según silhouette score: {optimal_k}")
        return optimal_k
    else:
        # Método simple del codo
        optimal_k = 3  # Valor por defecto conservador
        print(f"No se pudo determinar el número óptimo de clusters. Usando valor por defecto: {optimal_k}")
        return optimal_k

In [ ]:
def aplicar_kmeans_por_posicion(scaled_dfs, position_dfs, feature_cols):
    """
    Aplica KMeans a cada grupo posicional
    """
    kmeans_results = {}
    cluster_assignments = {}
    
    for position, df in scaled_dfs.items():
        if df.empty:
            print(f"No hay datos para posición: {position}")
            continue
        
        # Extraer características (sin jugador, equipo, etc.)
        X = df[feature_cols[position]].values
        
        # Determinar número óptimo de clusters
        print(f"\nAnalizando clusters óptimos para posición: {position}")
        k = determinar_kmeans_optimo(X, max_clusters=min(8, X.shape[0] - 1))
        
        # Aplicar KMeans con el número óptimo de clusters
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X)
        
        # Almacenar resultados
        kmeans_results[position] = kmeans
        
        # Agregar asignaciones de cluster al dataframe original
        df_with_clusters = position_dfs[position].copy()
        df_with_clusters['Cluster'] = clusters
        
        cluster_assignments[position] = df_with_clusters
        
        # Visualizar clusters usando PCA para reducción de dimensionalidad
        if X.shape[1] > 2:
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)
            
            plt.figure(figsize=(10, 8))
            scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', s=50, alpha=0.7)
            plt.colorbar(scatter, label='Cluster')
            
            # Agregar etiquetas para jugadores del Atlético de Madrid
            df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
            df_pca['Jugador'] = df['Jugador']
            df_pca['Equipo'] = df['Equipo']
            df_pca['Cluster'] = clusters
            
            atletico_players_pca = df_pca[df_pca['Equipo'] == 'Atletico Madrid']
            if not atletico_players_pca.empty:
                plt.scatter(atletico_players_pca['PC1'], atletico_players_pca['PC2'], 
                           color='red', marker='*', s=150, edgecolor='black', zorder=3)
                
                for _, player in atletico_players_pca.iterrows():
                    plt.annotate(player['Jugador'], 
                                (player['PC1'], player['PC2']),
                                xytext=(5, 5), textcoords='offset points',
                                fontsize=9, color='black')
            
            plt.title(f'Clustering de jugadores: {position}')
            plt.xlabel(f'PC1 (varianza: {pca.explained_variance_ratio_[0]:.2f})')
            plt.ylabel(f'PC2 (varianza: {pca.explained_variance_ratio_[1]:.2f})')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
            
            # Mostrar distribución de jugadores por cluster
            cluster_counts = pd.Series(clusters).value_counts().sort_index()
            
            plt.figure(figsize=(10, 6))
            bars = plt.bar(cluster_counts.index, cluster_counts.values)
            
            # Colorear barras según cluster
            cmap = plt.cm.get_cmap('viridis', k)
            for i, bar in enumerate(bars):
                bar.set_color(cmap(i))
            
            plt.title(f'Distribución de jugadores por cluster: {position}')
            plt.xlabel('Cluster')
            plt.ylabel('Número de jugadores')
            plt.xticks(cluster_counts.index)
            plt.grid(True, axis='y', linestyle='--', alpha=0.7)
            
            for i, count in enumerate(cluster_counts.values):
                plt.text(i, count + 0.5, str(count), ha='center')
                
            plt.tight_layout()
            plt.show()
        
    return kmeans_results, cluster_assignments

# Aplicar KMeans
kmeans_results, cluster_assignments = aplicar_kmeans_por_posicion(scaled_dfs, position_dfs, feature_cols)

In [ ]:
def analizar_clusters(cluster_assignments, feature_cols):
    """
    Analiza las características de cada cluster para entender sus perfiles
    """
    for position, df in cluster_assignments.items():
        print(f"\n{'='*20} Análisis de Clusters para {position} {'='*20}")
        
        # Características principales para esta posición
        features = feature_cols[position]
        
        # Análisis por cluster
        clusters = df['Cluster'].unique()
        
        for cluster in sorted(clusters):
            cluster_df = df[df['Cluster'] == cluster]
            print(f"\nCluster {cluster}: {len(cluster_df)} jugadores")
            
            # Jugadores representativos (top 5)
            print("Jugadores representativos:")
            representativos = cluster_df[['Jugador', 'Equipo', 'Nacionalidad']].head(5)
            display(representativos)
            
            # Perfil estadístico del cluster
            profile = cluster_df[features].describe().T[['mean', 'std', 'min', 'max']]
            display(profile)
            
            # Comparar con la media global
            global_mean = df[features].mean()
            cluster_mean = cluster_df[features].mean()
            
            # Calcular diferencia porcentual
            diff_pct = ((cluster_mean - global_mean) / global_mean * 100).fillna(0)
            
            # Mostrar características distintivas (las que más difieren de la media global)
            distinctive = diff_pct.abs().sort_values(ascending=False).head(5)
            
            print("\nCaracterísticas distintivas del cluster:")
            for feature in distinctive.index:
                direction = "por encima" if diff_pct[feature] > 0 else "por debajo"
                print(f"- {feature}: {abs(diff_pct[feature]):.1f}% {direction} de la media")
            
            # Identificar jugadores del Atlético en este cluster
            atletico_players = cluster_df[cluster_df['Equipo'] == 'Atletico Madrid']
            if not atletico_players.empty:
                print("\nJugadores del Atlético de Madrid en este cluster:")
                display(atletico_players[['Jugador', 'Nacionalidad']])
            
            print("-" * 50)

# Analizar los clusters resultantes
analizar_clusters(cluster_assignments, feature_cols)

## 4.2 Segundo Modelo (Knn para similitud de jugadores tras cluster relevante)

In [ ]:
def aplicar_knn_por_posicion(cluster_assignments, feature_cols, atletico_players):
    """
    Aplica KNN para encontrar jugadores similares a los del Atlético de Madrid
    """
    knn_results = {}
    
    for position, df in cluster_assignments.items():
        # Verificar si hay jugadores del Atlético en esta posición
        atletico_df = df[df['Equipo'] == 'Atletico Madrid']
        
        if atletico_df.empty:
            print(f"\nNo hay jugadores del Atlético de Madrid en posición {position}. Omitiendo KNN.")
            continue
        
        print(f"\n{'='*20} Análisis KNN para {position} {'='*20}")
        
        # Seleccionar características
        features = feature_cols[position]
        
        # Aplicar escalado estándar
        scaler = StandardScaler()
        X = scaler.fit_transform(df[features])
        
        # Crear índices para mapear dataframe original
        indices = pd.Series(df.index, index=df.index)
        
        # Crear modelo KNN
        knn = NearestNeighbors(n_neighbors=6)  # 6 = jugador + 5 similares
        knn.fit(X)
        
        # Almacenar resultados para esta posición
        position_results = []
        
        # Para cada jugador del Atlético, encontrar similares
        for idx, player in atletico_df.iterrows():
            # Obtener índice en el dataframe original
            player_idx = indices[idx]
            
            # Obtener vector de características del jugador
            player_features = X[player_idx].reshape(1, -1)
            
            # Encontrar vecinos más cercanos
            distances, indices_knn = knn.kneighbors(player_features)
            
            # Convertir índices a índices originales
            indices_original = [df.index[i] for i in indices_knn[0]]
            
            # Recuperar jugadores similares
            similar_players = df.loc[indices_original]
            
            # Excluir al propio jugador y otros jugadores del Atlético
            similar_players = similar_players[
                (similar_players.index != idx) & 
                (similar_players['Equipo'] != 'Atletico Madrid')
            ].head(5)
            
            # Almacenar resultados
            position_results.append({
                'jugador': player['Jugador'],
                'equipo': player['Equipo'],
                'cluster': player['Cluster'],
                'similares': similar_players
            })
            
            # Mostrar resultados
            print(f"\nJugador del Atlético: {player['Jugador']} (Cluster {player['Cluster']})")
            print("\nJugadores similares:")
            display(similar_players[['Jugador', 'Equipo', 'Nacionalidad', 'Cluster']])
            
            # Comparar características principales
            comparison = pd.DataFrame()
            player_series = pd.Series(player[features].values, index=features, name=player['Jugador'])
            comparison = pd.concat([comparison, player_series], axis=1)
            
            for idx_similar, similar in similar_players.iterrows():
                similar_series = pd.Series(similar[features].values, index=features, name=similar['Jugador'])
                comparison = pd.concat([comparison, similar_series], axis=1)
            
            # Visualizar comparación
            plt.figure(figsize=(12, 8))
            
            # Seleccionar subconjunto de características para visualización
            viz_features = features[:min(6, len(features))]  # Máximo 6 características para mejor visualización
            
            # Transponer para que cada jugador sea una serie
            comparison_viz = comparison.loc[viz_features].T
            
            # Crear radar chart
            angles = np.linspace(0, 2*np.pi, len(viz_features), endpoint=False).tolist()
            angles += angles[:1]  # Cerrar el círculo
            
            fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))
            
            # Normalizar datos para radar chart
            scaler_radar = MinMaxScaler()
            comparison_viz_scaled = pd.DataFrame(
                scaler_radar.fit_transform(comparison_viz),
                columns=comparison_viz.columns,
                index=comparison_viz.index
            )
            
            # Completar círculo para cada característica
            values_extended = {}
            for player_name in comparison_viz_scaled.index:
                values = comparison_viz_scaled.loc[player_name].values.flatten().tolist()
                values += values[:1]  # Cerrar el círculo
                values_extended[player_name] = values
            
            # Graficar jugador del Atlético
            atletico_values = values_extended[player['Jugador']]
            ax.plot(angles, atletico_values, 'o-', linewidth=2, label=player['Jugador'], color='red')
            ax.fill(angles, atletico_values, alpha=0.1, color='red')
            
            # Graficar jugadores similares
            colors = plt.cm.Set2(np.linspace(0, 1, len(similar_players)))
            for i, (similar_name, similar_values) in enumerate(values_extended.items()):
                if similar_name != player['Jugador']:
                    ax.plot(angles, similar_values, 'o-', linewidth=1.5, 
                           label=similar_name, color=colors[i-1])
                    ax.fill(angles, similar_values, alpha=0.05, color=colors[i-1])
            
            # Configurar gráfico
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(viz_features)
            ax.set_title(f'Comparación de {player["Jugador"]} con jugadores similares', size=15)
            ax.grid(True)
            plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
            plt.tight_layout()
            plt.show()
        
        knn_results[position] = position_results
    
    return knn_results

# Aplicar KNN para jugadores del Atlético de Madrid
knn_results = aplicar_knn_por_posicion(cluster_assignments, feature_cols, atletico_players)

# 5. Evaluación



In [ ]:
def evaluar_modelo(modelo, X_test, y_test):
    """
    Evaluar primer modelo
    """
    # Implementar evaluación
    pass

## 5.1 Evaluación de los resultados del Clustering

In [ ]:
def evaluar_clustering(kmeans_results, cluster_assignments, feature_cols):
    """
    Evalúa la calidad de los clusters generados
    """
    evaluation_results = {}
    
    for position, kmeans in kmeans_results.items():
        df = cluster_assignments[position]
        features = feature_cols[position]
        X = df[features].values
        
        print(f"\n{'='*20} Evaluación de clustering para {position} {'='*20}")
        
        # 1. Silhouette Score
        silhouette_avg = silhouette_score(X, df['Cluster'])
        print(f"Silhouette Score: {silhouette_avg:.4f}")
        
        # 2. Inercia (suma de distancias al cuadrado dentro de los clusters)
        inertia = kmeans.inertia_
        print(f"Inercia: {inertia:.4f}")
        
        # 3. Análisis de distribución de clusters
        cluster_distribution = df['Cluster'].value_counts().sort_index()
        print("\nDistribución de clusters:")
        for cluster, count in cluster_distribution.items():
            print(f"  Cluster {cluster}: {count} jugadores ({count/len(df)*100:.1f}%)")
        
        # 4. Homogeneidad por cluster (desviación estándar promedio de características)
        cluster_homogeneity = {}
        for cluster in df['Cluster'].unique():
            cluster_df = df[df['Cluster'] == cluster]
            std_avg = cluster_df[features].std().mean()
            cluster_homogeneity[cluster] = std_avg
        
        print("\nHomogeneidad por cluster (menor es mejor):")
        for cluster, homogeneity in cluster_homogeneity.items():
            print(f"  Cluster {cluster}: {homogeneity:.4f}")
        
        # 5. Centros de clusters
        print("\nCentros de clusters:")
        centers_df = pd.DataFrame(kmeans.cluster_centers_, columns=features)
        centers_df.index = [f'Cluster {i}' for i in range(len(centers_df))]
        display(centers_df.round(2))
        
        # Almacenar resultados
        evaluation_results[position] = {
            'silhouette_score': silhouette_avg,
            'inertia': inertia,
            'distribution': cluster_distribution,
            'homogeneity': cluster_homogeneity,
            'centers': centers_df
        }
    
    return evaluation_results

# Evaluar resultados del clustering
evaluation_results = evaluar_clustering(kmeans_results, cluster_assignments, feature_cols)

## 5.2 Evaluación de los resultados de KNN

In [ ]:
def evaluar_knn(knn_results, cluster_assignments, feature_cols):
    """
    Evalúa la calidad de los resultados de KNN para los jugadores del Atlético
    """
    if not knn_results:
        print("No hay resultados de KNN para evaluar")
        return
    
    for position, players_results in knn_results.items():
        print(f"\n{'='*20} Evaluación de KNN para {position} {'='*20}")
        
        df = cluster_assignments[position]
        features = feature_cols[position]
        
        for player_result in players_results:
            player = player_result['jugador']
            similares = player_result['similares']
            
            print(f"\nEvaluación para {player}:")
            
            # 1. Distribución de clusters de jugadores similares
            cluster_counts = similares['Cluster'].value_counts()
            print("Distribución de clusters entre jugadores similares:")
            for cluster, count in cluster_counts.items():
                print(f"  Cluster {cluster}: {count} jugadores")
            
            # 2. Distancia media a los jugadores similares
            player_row = df[df['Jugador'] == player]
            if player_row.empty:
                continue
                
            player_features = player_row[features].values[0]
            
            distances = []
            for _, similar in similares.iterrows():
                similar_features = similar[features].values
                distance = np.sqrt(np.sum((player_features - similar_features) ** 2))
                distances.append(distance)
            
            avg_distance = np.mean(distances)
            print(f"Distancia euclidiana media a jugadores similares: {avg_distance:.4f}")
            
            # 3. Variación en características clave
            key_features = features[:min(3, len(features))]  # Top 3 características
            
            similar_players_stats = similares[key_features]
            player_stats = player_row[key_features].values[0]
            
            print("\nVariación en características clave:")
            for i, feature in enumerate(key_features):
                variation = (similar_players_stats[feature] - player_stats[i]).abs().mean()
                print(f"  {feature}: Variación media = {variation:.4f}")
            
            print("-" * 50)

# Evaluar resultados de KNN
if knn_results:
    evaluar_knn(knn_results, cluster_assignments, feature_cols)

# 6. Despliegue

In [ ]:
def preparar_despliegue():
    """
    Preparar modelos para despliegue
    """
    # Implementar preparación
    pass

## 6.1 Preparación para Despliegue

In [ ]:
def preparar_despliegue(kmeans_results, cluster_assignments, knn_results):
    """
    Preparar modelos y resultados para su despliegue y uso
    """
    print("\n" + "="*20 + " PREPARACIÓN PARA DESPLIEGUE " + "="*20)
    
    # 1. Exportar resultados de clustering
    cluster_summary = {}
    
    for position, df in cluster_assignments.items():
        # Crear resumen de clusters
        cluster_counts = df['Cluster'].value_counts().to_dict()
        clusters_by_team = df.groupby(['Equipo', 'Cluster']).size().to_dict()
        atletico_clusters = df[df['Equipo'] == 'Atletico Madrid']['Cluster'].value_counts().to_dict()
        
        cluster_summary[position] = {
            'total_jugadores': len(df),
            'num_clusters': len(df['Cluster'].unique()),
            'distribucion_clusters': cluster_counts,
            'atletico_clusters': atletico_clusters
        }
    
    # Imprimir resumen de clustering
    print("\nResumen de clusters por posición:")
    for position, summary in cluster_summary.items():
        print(f"\n{position}:")
        print(f"  - Total jugadores: {summary['total_jugadores']}")
        print(f"  - Número de clusters: {summary['num_clusters']}")
        print("  - Distribución de clusters:")
        for cluster, count in summary['distribucion_clusters'].items():
            print(f"      Cluster {cluster}: {count} jugadores")
        
        print("  - Clusters de jugadores del Atlético:")
        if summary['atletico_clusters']:
            for cluster, count in summary['atletico_clusters'].items():
                print(f"      Cluster {cluster}: {count} jugadores")
        else:
            print("      No hay jugadores del Atlético en esta posición")
    
    # 2. Guardar resultados de KNN (jugadores similares)
    if knn_results:
        print("\nResumen de jugadores similares encontrados:")
        
        for position, players_results in knn_results.items():
            print(f"\n{position}:")
            
            for player_result in players_results:
                player = player_result['jugador']
                similares = player_result['similares']
                
                print(f"  - {player}: {len(similares)} jugadores similares identificados")
                
                # Mostrar top 3 más similares
                top3 = similares.head(3)
                print("    Top 3 más similares:")
                for _, similar in top3.iterrows():
                    print(f"      {similar['Jugador']} ({similar['Equipo']})")
    
    # 3. Instrucciones para integración
    print("\nInstrucciones para integración del modelo:")
    print("1. Los modelos de clustering (KMeans) están disponibles en la variable 'kmeans_results'")
    print("2. Para clasificar nuevos jugadores, usar la función de predicción del modelo correspondiente")
    print("3. Para encontrar similares a un nuevo jugador, aplicar el mismo preprocesamiento y usar KNN")
    
    return {
        'cluster_summary': cluster_summary,
        'kmeans_models': kmeans_results,
        'cluster_assignments': cluster_assignments,
        'knn_results': knn_results
    }

# Preparar para despliegue
deployment_package = preparar_despliegue(kmeans_results, cluster_assignments, knn_results)


## 6.2 Documentación de Uso

'''
TODO: Documentar:
- Cómo usar los modelos
- Requerimientos
- Limitaciones
- Mantenimiento necesario
'''

"""
# Documentación de Uso del Sistema de Análisis de Jugadores Similares

## Descripción general
Este sistema permite identificar jugadores con perfiles estadísticos similares utilizando técnicas de machine learning.
El análisis se realiza en dos fases:
1. Clustering (K-means): Agrupa jugadores con características similares
2. Análisis de similitud (KNN): Identifica jugadores específicamente similares a los del Atlético de Madrid

## Requerimientos técnicos
- Python 3.7+
- Bibliotecas: pandas, numpy, scikit-learn, matplotlib, seaborn
- Datos estadísticos actualizados (formato CSV)

## Cómo usar el sistema

### Para actualizar el análisis con nuevos datos
1. Colocar los archivos CSV actualizados en la carpeta del proyecto
2. Ejecutar el notebook completo para regenerar todos los análisis
3. Los resultados se mostrarán en las secciones de evaluación y despliegue

### Para clasificar un nuevo jugador
```python
# Cargar datos del nuevo jugador y preprocesar igual que en el entrenamiento
nuevo_jugador_df = pd.DataFrame(...)  # Datos del nuevo jugador
nuevo_jugador_features = nuevo_jugador_df[feature_cols[posicion]].values

# Escalar usando el mismo escalador usado en entrenamiento
scaler = StandardScaler()
X_train = cluster_assignments[posicion][feature_cols[posicion]].values
scaler.fit(X_train)
nuevo_jugador_scaled = scaler.transform(nuevo_jugador_features.reshape(1, -1))

# Predecir cluster
cluster = kmeans_results[posicion].predict(nuevo_jugador_scaled)[0]
print(f"El jugador pertenece al cluster {cluster}")
```

### Para encontrar jugadores similares a uno nuevo
```python
# Después de clasificar al jugador, usar KNN para encontrar similares
# Crear modelo KNN con los datos de entrenamiento
knn = NearestNeighbors(n_neighbors=6)
X = scaler.transform(X_train)
knn.fit(X)

# Encontrar vecinos más cercanos
distances, indices = knn.kneighbors(nuevo_jugador_scaled)
similares = cluster_assignments[posicion].iloc[indices[0]]
print("Jugadores similares:")
print(similares[['Jugador', 'Equipo', 'Nacionalidad']])
```

## Limitaciones
- El rendimiento del modelo depende de la calidad y actualización de los datos
- Las métricas utilizadas están adaptadas a cada posición pero podrían refinarse
- La similitud se basa en estadísticas, no considera factores cualitativos o tácticos
- El análisis es más preciso con mayor cantidad de minutos jugados

## Mantenimiento necesario
- Actualizar datos al menos cada 3-4 jornadas para mantener relevancia
- Revisar y ajustar características por posición según evolución del juego
- Validar periódicamente los resultados con expertos del departamento deportivo
"""

# 7. Conclusiones y Recomendaciones
'''
TODO: Documentar:
- Resultados principales
- Interpretación deportiva
- Limitaciones encontradas
- Mejoras propuestas
- Aplicaciones prácticas
'''

## Resultados principales

### Clustering
- Se han identificado grupos de jugadores con perfiles estadísticos diferenciados por posición
- Los clusters muestran coherencia deportiva, agrupando jugadores de características similares
- La distribución de clusters varía según la posición, reflejando la diversidad de perfiles

### Análisis KNN
- Para cada jugador del Atlético de Madrid, se han encontrado jugadores estadísticamente similares
- La mayoría de jugadores similares pertenecen al mismo cluster, validando la consistencia del análisis
- Se han identificado alternativas en diferentes ligas y equipos para cada perfil

## Interpretación deportiva
- Los centrales del Atlético presentan un perfil defensivo sólido, destacando en duelos aéreos y despejes
- Los laterales muestran un equilibrio entre fases defensiva y ofensiva, con capacidad de progresión
- Los centrocampistas se dividen entre perfiles más creativos y otros más defensivos
- Los delanteros se caracterizan por su eficiencia en área, aunque con diferentes perfiles de movilidad

## Limitaciones encontradas
- La calidad del análisis depende de la disponibilidad y calidad de los datos estadísticos
- Algunas métricas importantes pueden no estar capturadas en los datos disponibles
- El análisis puramente estadístico no captura aspectos tácticos o de comportamiento
- La temporalidad de los datos puede afectar a la comparabilidad (forma reciente vs. histórica)

## Mejoras propuestas
- Incorporar datos de tracking para análisis de movimientos sin balón
- Añadir métricas contextuales (rivales, resultado, sistema táctico)
- Implementar análisis de video para complementar métricas estadísticas
- Crear un sistema de ponderación variable según necesidades específicas del club
- Desarrollar una interfaz visual interactiva para el departamento técnico

## Aplicaciones prácticas
- Identificación de objetivos de mercado alineados con el perfil actual del equipo
- Análisis de debilidades y fortalezas tácticas por comparación con perfiles similares
- Detección temprana de talentos emergentes con características similares a referentes
- Planificación de plantilla a largo plazo, identificando posibles reemplazos
- Preparación táctica contra rivales, conociendo jugadores con perfiles similares a los propios

El modelo desarrollado proporciona una base sólida para la toma de decisiones basada en datos
dentro del departamento deportivo, complementando el análisis tradicional con insights
cuantitativos y objetivos.
"""

In [ ]:
# Ejecución principal
if __name__ == "__main__":
    print("Análisis de jugadores similares mediante Clustering finalizado")


if __name__ == "__main__":
    # Ejecución principal
    print("Iniciando análisis...")